In [1]:
from line_solver import *

In [2]:
model = Network('model')

P1 = Place(model, 'P1')
P2 = Place(model, 'P2')
T1 = Transition(model, 'T1')
T2 = Transition(model, 'T2')

jobclass = ClosedClass(model, 'Class1', 10, P1, 0)

In [3]:
# T1 - Batch processing: requires 4 tokens, produces 4 tokens
mode1 = T1.add_mode('Mode1')
T1.setDistribution(mode1, Exp(2))
T1.setEnablingConditions(mode1, jobclass, P1, 4)  # Requires 4 tokens from P1
T1.setFiringOutcome(mode1, jobclass, P2, 4)       # Produces 4 tokens to P2

# T2 - Batch processing: requires 2 tokens, produces 2 tokens
mode2 = T2.add_mode('Mode2')
T2.setDistribution(mode2, Exp(3))
T2.setEnablingConditions(mode2, jobclass, P2, 2)  # Requires 2 tokens from P2
T2.setFiringOutcome(mode2, jobclass, P1, 2)       # Produces 2 tokens to P1

In [4]:
# Set up routing matrix
routingMatrix = model.init_routing_matrix()
routingMatrix.set(jobclass, jobclass, P1, T1, 1.0)  # P1 -> T1
routingMatrix.set(jobclass, jobclass, P2, T2, 1.0)  # P2 -> T2
routingMatrix.set(jobclass, jobclass, T1, P2, 1.0)  # T1 -> P2
routingMatrix.set(jobclass, jobclass, T2, P1, 1.0)  # T2 -> P1

model.link(routingMatrix)

In [5]:
# Set initial state - all 10 tokens start in P1
P1.setState([jobclass.getPopulation()])
P2.setState([0])

In [6]:
# Solve the model
solver = SolverJMT(model, seed=23000)
avgTable = solver.get_avg_table()

JMT Model: /tmp/workspace/jsim/8968712463160524897/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.171107s.
  Station JobClass    QLen  Util   RespT  ResidT    ArvR    Tput
0      P1   Class1  4.6632   0.0  0.8976  0.8976  5.0348  4.9991
1      P2   Class1  5.3368   0.0  1.0474  1.0474  4.9991  5.0357
